In [2]:
import pandas as pd
import os
import librosa
import numpy as np
import tensorflow
import keras

#from Documents.helpers.wavfilehelper import WavFileHelper


In [6]:
import struct

class WavFileHelper():
    
    def read_file_properties(self, filename):

        wave_file = open(filename,"rb")
        
        riff = wave_file.read(12)
        fmt = wave_file.read(36)
        
        num_channels_string = fmt[10:12]
        num_channels = struct.unpack('<H', num_channels_string)[0]

        sample_rate_string = fmt[12:16]
        sample_rate = struct.unpack("<I",sample_rate_string)[0]
        
        bit_depth_string = fmt[22:24]
        bit_depth = struct.unpack("<H",bit_depth_string)[0]

        return (num_channels, sample_rate, bit_depth)
wavfilehelper = WavFileHelper()    

In [7]:
audiodata = []
# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'G:/urban_dataset'

metadata = pd.read_csv(fulldatasetpath + '/metadata/UrbanSound8K.csv')


for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath('G:/urban_dataset/'),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    data = wavfilehelper.read_file_properties(file_name)
    audiodata.append(data)

# Convert into a Panda dataframe
audiodf = pd.DataFrame(audiodata, columns=['num_channels','sample_rate','bit_depth'])

In [9]:
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') # default is Kaiser_best
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [10]:
# Set the path to the full UrbanSound dataset 
fulldatasetpath = 'G:/urban_dataset'

metadata = pd.read_csv(fulldatasetpath + '/metadata/UrbanSound8K.csv')

features = []
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files')

Finished feature extraction from  8732  files


In [11]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [12]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout,Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import model_from_json

In [14]:
# neural network dimensions
n_dim =x_train.shape[1]
n_classes = y_train.shape[1]
n_hidden_units_1 = n_dim
n_hidden_units_2 = 400 # approx n_dim * 2
n_hidden_units_3 = 200 # half of layer 2

In [15]:
print(n_dim)
print("Features: {} Classes: {} ".format(n_dim, n_classes))

40
Features: 40 Classes: 10 


In [16]:
def create_model(activation_function='relu',init_type='normal', kernel_initializer="uniform", optimiser='Adamax', dropout_rate=0.5):
    model = Sequential()
    # layer 1
    #model.add(Convolution2D(n_hidden_units_1, input_dim=n_dim, activation=activation_function,kernel_initializer="uniform"))
    #model.add(MaxPooling2D(2,2))
    model.add(Dense(n_hidden_units_1, input_dim=n_dim, init=init_type, activation=activation_function))
    # layer 2
    model.add(Dense(n_hidden_units_2, kernel_initializer="uniform", activation=activation_function))
    model.add(Dropout(dropout_rate))
    # layer 3
    model.add(Dense(n_hidden_units_3, kernel_initializer="uniform" , activation=activation_function))
    model.add(Dropout(dropout_rate))
    # output layer
    model.add(Dense(n_classes, kernel_initializer="uniform", activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer=optimiser, metrics=['accuracy'])
    return model

In [17]:
model = create_model()
earlystop = EarlyStopping(monitor='val_loss', patience=3, verbose=0, mode='auto')

C:\Users\Admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(40, input_dim=40, activation="relu", kernel_initializer="normal")`
  


In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_2 (Dense)              (None, 400)               16400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               80200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2010      
Total params: 100,250
Trainable params: 100,250
Non-trainable params: 0
________________________________________________

In [20]:
history = model.fit(x_train, y_train, epochs=30, validation_split=0.1, callbacks=[earlystop], batch_size=24)

Train on 6286 samples, validate on 699 samples
Epoch 1/30
6286/6286 [==============================] - 3s 516us/step - loss: 1.8261 - acc: 0.3392 - val_loss: 1.4978 - val_acc: 0.4993
Epoch 2/30
6286/6286 [==============================] - 3s 418us/step - loss: 1.4312 - acc: 0.5060 - val_loss: 1.2692 - val_acc: 0.5908
Epoch 3/30
6286/6286 [==============================] - 2s 339us/step - loss: 1.2711 - acc: 0.5681 - val_loss: 1.1549 - val_acc: 0.6223
Epoch 4/30
6286/6286 [==============================] - 2s 334us/step - loss: 1.1750 - acc: 0.6090 - val_loss: 1.0848 - val_acc: 0.6495
Epoch 5/30
6286/6286 [==============================] - 2s 302us/step - loss: 1.0848 - acc: 0.6419 - val_loss: 1.0389 - val_acc: 0.6652
Epoch 6/30
6286/6286 [==============================] - 2s 321us/step - loss: 0.9956 - acc: 0.6699 - val_loss: 0.9789 - val_acc: 0.6795
Epoch 7/30
6286/6286 [==============================] - 2s 295us/step - loss: 0.9426 - acc: 0.6909 - val_loss: 0.9390 - val_acc: 0.6938
E

In [ ]:
score, accuracy = model.evaluate(x = x_test, y = y_test , verbose = 1)
print("ACCURACY:",(accuracy*100))
print("SCORE:",score)

In [24]:
model.save("nn_mfcc_model.h5")
print("Saved model to disk")

Saved model to disk


In [25]:
from numpy import loadtxt
from keras.models import load_model
 
# load model
model = load_model('nn_mfcc_model.h5')
# summarize model.
model.summary()
# load dataset

# evaluate the model
score = model.evaluate(x=x_test,y=y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 40)                1640      
_________________________________________________________________
dense_2 (Dense)              (None, 400)               16400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               80200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2010      
Total params: 100,250
Trainable params: 100,250
Non-trainable params: 0
________________________________________________